<a href="https://colab.research.google.com/github/ahmedoumar/TimeSerieswithTf/blob/main/TimeSeriesWithTf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Uploading training data:
from google.colab import files
uploaded = files.upload()

    

Saving Opt_CL.csv to Opt_CL.csv
Saving Opt_6B.csv to Opt_6B.csv
Saving Opt_NQ.csv to Opt_NQ.csv
Saving Opt_FDAX.csv to Opt_FDAX.csv
Saving Opt_RTY.csv to Opt_RTY.csv
Saving Opt_YM.csv to Opt_YM.csv
Saving Opt_ZN.csv to Opt_ZN.csv


In [ ]:
# Import our main packages:
import os
import datetime
from sklearn.preprocessing import StandardScaler
import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv1D, Input, BatchNormalization, ReLU, GlobalAveragePooling1D, Dense
from tensorflow.keras.models import Model

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [ ]:
!ls

Opt_6B.csv  Opt_FDAX.csv  Opt_RTY.csv  Opt_ZN.csv
Opt_CL.csv  Opt_NQ.csv	  Opt_YM.csv   sample_data


In [ ]:
df = pd.read_csv('Opt_6B.csv')
df.head()

,Profit,Entry Name,H1,L1,O1,C1,V1,HiLo1,H2,L2,O2,C2,V2,HiLo2,H3,L3,O3,C3,V3,HiLo3,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11,I12,I13,I14,I15,I16,I17,I18,I19,I20,I21,I22,I23,I24,I25,I26,I27,I28,I29,I30,I31
0,0,R1 Sell,1.5924,1.5922,1.5924,1.5924,70,1.592250,1.5928,1.5924,1.5926,1.5924,14,1.592225,1.5926,1.5923,1.5925,1.5926,52,1.592075,1,0.0006,8.988355,33.833333,0.000553,0,5.691768,0.000025,1,53.037985,0,-0.174,-28149,5.691768,0.200000,1,55.109071,90,0,61.668096,34500,0,52.018616,-36.363636,1.089286,1,9.940381,0.602561,0,9.146679,Friday
1,0,R1 Buy,1.5939,1.5923,1.5938,1.5931,1447,1.593950,1.5940,1.5923,1.5930,1.5940,1738,1.594425,1.5936,1.5925,1.5932,1.5931,1321,1.595050,0,-0.0044,-5.944462,-115.160719,0.001409,0,29.576539,-0.000288,1,44.849324,1,-0.177,-29712,29.576539,-0.166667,1,59.239261,20,1,47.234003,104500,0,36.124563,-88.405797,0.950000,1,-18.540868,0.017070,0,31.143836,Friday
2,0,R1 Sell,1.5887,1.5879,1.5883,1.5886,75,1.587925,1.5888,1.5880,1.5885,1.5884,151,1.588575,1.5885,1.5880,1.5880,1.5885,72,1.588525,1,0.0001,5.477516,-35.745730,0.001054,1,12.451115,-0.000105,0,50.533011,0,-0.093,-45651,12.451115,-0.666667,1,50.042168,0,0,47.906478,21500,1,60.676506,-72.972973,1.083969,0,0.802873,0.309199,1,14.563587,Monday
3,0,R1 Sell,1.5901,1.5887,1.5895,1.5891,865,1.588975,1.5906,1.5894,1.5902,1.5895,584,1.589100,1.5902,1.5883,1.5896,1.5902,694,1.589075,0,0.0004,-1.657371,-14.736842,0.001039,0,15.030060,0.000066,1,48.935102,1,-0.092,-45650,15.030060,0.063830,0,57.870256,70,0,60.985333,91500,1,54.949556,-65.517241,0.993103,0,-1.106353,0.129379,1,19.139527,Monday
4,0,R1 Buy,1.5899,1.5888,1.5897,1.5891,341,1.590200,1.5899,1.5889,1.5898,1.5896,272,1.590350,1.5906,1.5898,1.5903,1.5899,335,1.590575,0,-0.0007,-5.305886,-115.950920,0.001267,1,12.575675,-0.000007,0,47.131680,1,-0.020,-44267,12.575675,-0.650000,0,59.650944,40,1,73.735046,164500,0,50.414254,-78.260870,1.012422,1,-7.843250,0.025864,0,16.329333,Monday


In [ ]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Profit,5278.0,0.322471,0.467466,0.000000,0.000000,0.000000,1.000000,1.000000
H1,5278.0,1.498811,0.156461,1.162100,1.343800,1.525150,1.634400,1.783400
L1,5278.0,1.497950,0.156485,1.159100,1.343000,1.524150,1.633700,1.782800
O1,5278.0,1.498383,0.156471,1.162100,1.343400,1.524700,1.633975,1.783300
C1,5278.0,1.498373,0.156469,1.160000,1.343300,1.524850,1.633900,1.782900
V1,5278.0,852.253695,1111.504892,1.000000,152.000000,510.000000,1208.750000,24393.000000
HiLo1,5278.0,1.498338,0.156480,1.162075,1.343275,1.523687,1.633956,1.782750
H2,5278.0,1.498810,0.156460,1.162000,1.343700,1.525000,1.634375,1.783800
L2,5278.0,1.497954,0.156483,1.158900,1.342900,1.523850,1.633475,1.783200
O2,5278.0,1.498385,0.156469,1.158900,1.343325,1.524200,1.633800,1.783300


In [ ]:
# Drop unused columns 
df.drop(['Entry Name', 'I31'], inplace=True, axis=1)

# Make data transformation:
columns = list(df.keys())
df[columns[1:]] = StandardScaler().fit_transform(df[columns[1:]])

# Splitting the data into train/val: 80%/20%
columns_indices = {name: i for i, name in enumerate(df.columns)}

n = len(df)
train_df = df[0: int(n*.8)]
val_df = df[int(n*.8):]

num_features = df.shape[1]

In [ ]:
print(num_features)

49


In [ ]:
# Build Our Model:

def make_model(input_shape):
    input_layer = Input(input_shape)

    conv1 = Conv1D(filters=64, kernel_size=3, padding='same')(input_layer)
    conv1 = BatchNormalization()(conv1)
    conv1 = ReLU()(conv1)

    conv2 = Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
    conv2 = BatchNormalization()(conv2)
    conv2 = ReLU()(conv2)

    conv3 = Conv1D(filters=64, kernel_size=3, padding="same")(keras.layers.concatenate([conv2, conv1]))
    conv3 = BatchNormalization()(conv3)
    conv3 = ReLU()(keras.layers.concatenate([conv3,conv2]))

    gap = GlobalAveragePooling1D()(conv3)

    output_layer = Dense(1, activation="sigmoid")(gap)

    return Model(inputs=input_layer, outputs=output_layer)

In [ ]:
y_train = train_df['Profit']
train_df.drop('Profit', inplace=True, axis=1)

y_val = val_df['Profit']
val_df.drop('Profit', inplace=True, axis=1)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
train_df.shape

(4222, 48)

In [ ]:
x_train = np.asarray(train_df).astype(np.float32)
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))

In [ ]:
x_val = np.asarray(val_df).astype(np.float)
x_val = x_val.reshape((x_val.shape[0], x_val.shape[1], 1))

print(x_val.shape)

(1056, 48, 1)


In [ ]:
y_train = np.asarray(y_train).astype(np.float32)
y_train = y_train.reshape((y_train.shape[0], 1))

print(y_train.shape)

y_val = np.asarray(y_val).astype(np.float)
y_val = y_val.reshape((y_val.shape[0], 1))

print(y_val.shape)

(4222, 1)
(1056, 1)


In [ ]:
model1 = make_model(input_shape=x_train.shape[1:])
#tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
epochs = 500
batch_size = 128

callbacks = [
             keras.callbacks.ModelCheckpoint(
                 'best_model.h5', save_best_only=True,
                monitor='val_recall'
             ),
             keras.callbacks.ReduceLROnPlateau(
                 monitor='val_recall', factor=.2,
                 patience=100, min_lr = .0001
             ),
             keras.callbacks.EarlyStopping(
                 monitor='val_recall', patience=100
             )
]

model1.compile(
    optimizer=keras.optimizers.Adam(0.0001),
    loss='binary_crossentropy',
    metrics=['binary_accuracy', keras.metrics.Recall()],
    
)


In [ ]:
history = model1.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    verbose=1,
    validation_data = (x_val, y_val),
    class_weight = {0:1, 1:2.1}
)